In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.special import gammainc

In [4]:
def sigma(sig_ln, F0, alpha):
    ''' Compute sigma'''
    return sig_ln * F0**(1-alpha)

def X(F, alpha, sig):
    ''' X transform variable '''
    return F**(2 * (1-alpha)) / (sig**2 * (1-alpha)**2)

def F_Xinv(X, alpha, sig):
    ''' F from X'''
    return (X * (sig**2 * (1-alpha)**2)) ** (1/(2*(1-alpha)))

def delta(alpha):
    ''' delta from alpha'''
    return (1 - 2*alpha) / (1 - alpha)

def em_CEV(T, dt, F0, alpha, sig):
    ''' Euler scheme simulation '''
    
    # set up Brownian motion and t
    t = np.arange(dt, T + dt, dt)
    N = len(t)
    dW=np.sqrt(dt)*np.random.randn(N)
    
    # init variables
    X0 = X(F0, alpha, sig)
    d = delta(alpha)
    X_emC, X_em = X0, []
    hit_zero = False
    
    # For all timesteps
    for j in range(N):
        # euler (X + XdX)=(X + X * (d*dt + 2X^(1/2)dW))
        dX = d*dt + 2*np.sqrt(X_emC)*dW[j]
        X_emC += dX
        
        # if <= 0, presumed it hit boundary of 0
        if X_emC <= 0:
            # fill with 0's since it is absorbed
            X_em += [0 for i in range(N - len(X_em))]
            hit_zero = True
            break
        X_em.append(X_emC)
    
    return X_em, hit_zero



def mil_CEV(T, dt, F0, alpha, sig):
    ''' Milstein scheme simulation '''
    
    # set up Brownian motion and t
    t = np.arange(dt, T + dt, dt)
    N = len(t)
    dW=np.sqrt(dt)*np.random.randn(N)
    
    # init variables
    X0 = X(F0, alpha, sig)
    d = delta(alpha)
    X_milC, X_mil = X0, []
    hit_zero = False
    
    # For all timesteps
    for j in range(N):
        # Milstein 
        dX = d*dt + 2*np.sqrt(X_milC)*dW[j] + (dW[j]**2 - dt)
        X_milC += dX
        
        # if <= 0, presumed it hit boundary of 0
        if X_milC <= 0:
            # fill with 0's since it is absorbed
            X_mil += [0 for i in range(N - len(X_mil))]
            hit_zero = True
            break
        X_mil.append(X_milC)
    
    return X_mil, hit_zero
    
    

In [5]:
def analytical_C(F,K,T, alpha, sig):
    K_tilde = X(K,alpha, sig)
    X0 = X(F,alpha, sig)
    C = F * (1 - stats.ncx2.cdf(x = K_tilde/T, df = 4 - delta(alpha), nc = X0/T)) - K * stats.ncx2.cdf(x = X0/T, df = 2 - delta(alpha),nc = K_tilde/T)
    
    return C

In [21]:
from scipy.optimize import root
import tqdm
# Parameters
F = 100
alpha = 0
sig_ln = 0.5
T = 4
K = 90
# transform sig to correct dimension
sig = sigma(sig_ln, F, alpha)
# print(sig)

# Umax
v = delta(alpha)/2 - 1
X0 = X(F, alpha, sig)
# print(X0)
Umax = gammainc(-v, X0/(2*T))

# U
s = stats.qmc.Sobol(1)
M = 10
N = 2**M
U = s.random_base2(M).flatten()


absorbed = U > Umax
print(f"Sim Absorbed: {np.count_nonzero(absorbed)/N}")
print(f"Ana Absorbed: {1 - gammainc(-v, X0/(2*T))}")


XoverT = np.zeros(N)
not_absorbed = ~absorbed
# print(X0/T)
def func(x, u):
    return stats.ncx2.cdf(X0/T, df = 2 - delta(alpha), nc = x/T) - Umax + u
for idx, u in tqdm.tqdm(enumerate(U), total = N):
    if u > Umax:
        continue
    else:
        sol = root(func, X0, args=(u))
        XoverT[idx] = sol.x[0]

X_T = XoverT

# compute inverse to get F_T
F_T = F_Xinv(X_T, alpha, sig)
# compute C by subtracting K and taking max. Anywhere less than K is 0, otherwise F - K
C = np.where(F_T<K, 0, F_T-K)
P = C - (F - K)


print(C.mean())
print(P.mean())
print(analytical_C(F,K,T,alpha,sig))

Sim Absorbed: 0.3173828125
Ana Absorbed: 0.31731050786291415


100%|██████████| 1024/1024 [00:00<00:00, 1024.75it/s]

44.28051044926829
34.28051044926829
43.98809800804455


In [34]:
print(X0/T)
print(delta(alpha))
f = lambda x, u: stats.ncx2.cdf(X0/T, df = 2 - delta(alpha), nc = x/T) + Umax - u

1.0
1.0


In [14]:
print(Umax)

0.6826894921370859


In [13]:
print(u)

0.6543200481683016


In [68]:
def func(x, u):
    return stats.ncx2.cdf(X0/T, df = 2 - delta(alpha), nc = x/T) - Umax + u

In [76]:
func(50.87, u)

1.0976647153326802e-05

In [77]:
root(func, X0, args = (u))

    fjac: array([[-1.]])
     fun: array([0.])
 message: 'The solution converged.'
    nfev: 15
     qtf: array([-1.30229161e-13])
       r: array([0.00051809])
  status: 1
 success: True
       x: array([50.89116456])

In [37]:
F

100

In [44]:

f(10000, u)

0.00512810285474008